# Module 10 Assignment

A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `YOUR CODE HERE`. Do **not** write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed or overwritten by the autograder.

2. Before you submit your assignment, make sure everything runs as expected. Go to menubar, select _Kernel_, and restart the kernel and run all cells (_Restart & Run all_).

3. Do not change the title (i.e. file name) of this notebook.

4. Make sure that you save your work (in the menubar, select _File_ → _Save and CheckPoint_)

5. You are allowed to submit an assignment multiple times, but only the most recent submission will be graded.

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
from nose.tools import assert_equal, assert_is_instance, assert_almost_equal, assert_true
import pymc3 as pm
# These two lines suppress warnings that sometimes
# occur when making visualizations
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder


Just as in the $\texttt{intro2pp}$ notebook we will first generate some data from a normal distribution with mean 0 and standard deviation 1

In [ ]:
# Define random state for reproducability
rng = np.random.RandomState(23)

# Generate normal data
mu = 0.0
sigma = 1.0
size = 5000
data = rng.normal(mu, sigma, size)

# Problem 1: Estimate model parameters

Write a function called $\texttt{generate_trace}$ which uses PyMC3 to estimate the parameters of a model. Assume that the data are generated from a $N(\mu,\sigma^{2})$ and with prior distributions $\mu \sim N(\alpha,\gamma^{2})$ and $\sigma \sim U(a,b)$. Label the prior for $\mu$ as "mean" and the prior for $\sigma$ as "sigma".

In [ ]:
def generate_trace(n_samples,data,alpha,gamma,a,b):
    '''
    Inputs
    -------
    n_samples: the number of samples to generate
    
    data: the observed data
    
    alpha: the mean for the Normal prior on the mean
    
    gamma: the standard deviation for the Normal prior on the mean
    
    a: the lower bound on the Uniform prior on the standard deviation
    
    b: the upper bound on the Uniform prior on the standard deviation
    
    Returns
    -------
    
    trace: the generated samples for the mean and standard deviation
    
    '''
    
    # YOUR CODE HERE
        
    return trace

In [ ]:
trace = generate_trace(10000,data,0,1,0,10)

In [ ]:
assert_is_instance(trace,pm.backends.base.MultiTrace)
assert_almost_equal(len(trace.get_values('mean')),10000)
assert_almost_equal(len(trace.get_values('sigma')),10000)


In [ ]:
#Let's plot the traces and see how they look, just like in the intro2pp notebook
fig, axs = plt.subplots(figsize=(12, 4), 
                        nrows=2, ncols=2, 
                        gridspec_kw = {'width_ratios':[1, 3]})

_ = pm.traceplot(trace, ax=axs)

Just as in the $\texttt{intro2pp-bm}$ notebook we will first generate some data from a linear model.

In [ ]:
m_true = 0.5
b_true = 0.25

np.random.seed(23)

def theModel(xmin=0, xmax = 1, num=20):
    
    sigma = 0.1

    x = np.linspace(xmin, xmax, num)
    y = b_true + m_true * x - sigma * np.random.randn(len(x))

    return(x, y)



x, y = theModel(num = 50)


# Problem 2: Estimate model parameters

Write a function called $\texttt{generate_trace}$ which uses PyMC3 to estimate the parameters of a model. Assume that the data are generated by the equation $y=\beta_{0} + \beta_{1}x - \sigma\epsilon$ where $\epsilon$ is a random number which represents noise. Suppose that $\beta_{0}$ and $\beta_{1}$ have a $N(\mu,s^{2})$ prior and $\sigma$ has a $N(\alpha,\gamma^{2})$ prior. Label the prior for $\beta_{0}$ as "intercept", the prior for $\beta_{1}$ as "slope", and the prior for $\sigma$ as "sigma". Furthermore, unlike in the $\texttt{intro2pp-bm}$ notebook, do not specify a start or a step argument in the PyMC3 sampler.

In [ ]:
def fit_model(n_samples,data,mu,sd,alpha,gamma):
    '''
    Inputs
    -------
    n_samples: the number of samples to generate
    
    data: the observed data
    
    mu: the mean of the Normal prior for the slope and the intercept
    
    sd: the standard deviation of the Normal prior for the slope and the intercept
    
    alpha: the mean for the Normal prior on the error term
    
    gamma: the standard deviation for the Normal prior on the error term
    

    Returns
    -------
    
    trace: the generated samples for the mean and standard deviation
    
    '''
    
    # YOUR CODE HERE
        
    return trace

In [ ]:
trace = fit_model(1000,y,0,1,0,10)

In [ ]:
trace.varnames

In [ ]:
assert_is_instance(trace,pm.backends.base.MultiTrace)
assert_almost_equal(len(trace.get_values('Intercept')),1000)
assert_almost_equal(len(trace.get_values('Slope')),1000)
assert_almost_equal(len(trace.get_values('sigma')),1000)
assert_equal(1, "Intercept" in trace.varnames)
assert_equal(1, "Slope" in trace.varnames)
assert_equal(1, "sigma" in trace.varnames)

In [ ]:
#Let's plot the traces and see how they look, just like in the intro2pp notebook
fig, axs = plt.subplots(figsize=(12, 4), 
                        nrows=3, ncols=2, 
                        gridspec_kw = {'width_ratios':[1, 3]})

_ = pm.traceplot(trace, ax=axs)

# Problem 3: Get samples after burn in

Write a function called $\texttt{burn_in}$ which given a PyMC3 trace and a number of burn in samples, returns the samples of the parameters from the previous question after applying burn in. The function should return a tuple, where the first element has the trace values of the intercept, the second elements has the trace values of the slope, and the third element has the trace values for sigma.

In [ ]:
def burn_in(trace,burn_in):
    '''
    Inputs
    -------
    trace: the sampled parameter values from the previous Bayesian model
    
    burn_in: the number of samples to burn in
    
    Returns
    -------
    
    vals_after_burnin: a tuple, the sampled values of each unknown parameter after burn in
    '''
    
    # YOUR CODE HERE
    
    return vals_after_burnin

In [ ]:
i, sl, si = burn_in(trace,0)
assert_equal(len(i),1000)
i, sl, si = burn_in(trace,500)
assert_equal(len(sl),500)

In [ ]:
np.random.seed(23)

# Problem 4: Preprocessing Data

Read in the imports-85 dataset for the label use the price column. For the features use all columns (excluding price) that are not objects. Now run PCA on the features and keep only 1 feature and set the random_state tp be 20. Use 2 StandardScalers in scikit-learn to fit and transform the features and labels (name your transformed features as x and your transformed labels as y). Finally save your features and labels in a dataframe called df with the column names x and y respectively.


In [ ]:
# YOUR CODE HERE

In [ ]:
assert_equal(df.columns.tolist(), ['x', 'y'])
assert_equal(df['x'].values.tolist(),
[-0.014354147645092491, -0.014354147645093141, 0.515393110115907, -0.420236745065213, 0.516109917862961, -0.09144645288664967, 0.555051685687836, 0.7652805703339163, 1.0179455430777125, 0.9535952354511762, -0.3089243626064649, -0.30892086535862545, 0.30159399879312426, 0.406915592617665, 0.9621055703505771, 1.303219760177122, 1.5906478469166971, 1.8303384423431908, -2.0552919347506955, -1.3115340476751471, -1.24458811179575, -1.3073762828705155, -1.307206337242126, -0.8233192597426621, -1.1326049229338204, -1.090476113103875, -1.090476113103875, -0.7023352721737748, -0.040019698757362665, 0.4933389099508201, -1.6201368378749361, -1.4165057123715605, -1.3840088221766624, -1.1847710039152868, -1.154132407003555, -1.050034229702658, -1.023832584153444, -0.6147204142398616, -0.5132300619697507, -0.4844280218409981, -0.3542324549313596, -0.17597145727540822, -0.5054190996328098, -0.4207966225742045, -1.3112568782105631, -1.2442324903719995, 0.3405873740990335, 2.911421803128229, 2.9111584478903674, 2.6974751064331697, -1.280193635350738, -1.2612041389577382, -1.2516295774228219, -1.174621303340197, -1.165046741805281, -0.34327256237343445, -0.34327256237343445, -0.33369800083851836, -0.1121599272978253, -0.3270715082239262, -0.2793510429705258, -0.3270715082239262, -0.2793510429705258, -0.21643240663937438, -0.2506280974420922, 0.22112435384442042, 0.27740448983334304, 1.84532243884707, 2.2953333934603077, 1.806826898701236, 2.3341961921192595, 2.2839580229026955, 2.1776221186248454, 2.600386220345794, 2.2450958079938834, 0.680919214932278, -1.2261897863906257, -1.1762320987655093, -1.0613373603465157, -0.7902816285640626, -0.3572470765776134, -0.4361821314015558, 0.5352074169528312, 0.7037196999673548, 0.7132942615022712, -0.36566481156230163, -0.28906831928297266, -0.2943894059662922, -0.29424992292611984, -1.280917191970833, -1.0355154927243875, -1.2253847350683198, -1.1871537733545905, -1.022495684824729, -1.1621926289378735, -1.0147363523429165, -1.123961667224144, -0.9976018248339472, -1.0526925087205352, -0.4446686508972363, -0.48679598257455253, 1.0407626404123103, 1.425549858471694, 0.9735301124568825, 0.9948590677072959, 1.1251473184493164, 1.1259451579856299, 0.8898295577237234, 1.2326269380948305, 1.29211462045765, 1.6791712664473875, 0.9951497346078008, 1.3379471149789077, 1.3974295033454853, 1.784491443331465, 0.9951497346078008, 1.3379471149789077, 1.1023111411892188, -1.2269331213557328, -0.823302393106011, -1.132530805158504, -1.0900033182186486, -0.7021470880481833, -0.0404264905468373, 0.5064853572707578, 0.42973307978396696, 0.39245767491947037, 0.39245767491947037, 0.4767108601214738, 1.5621396940345913, 0.04583451672962686, -0.1822439094440613, 0.19634768849639692, 0.2666759075343549, 0.2901783695234114, 0.3873153828742982, 0.4836607077605772, 0.5578187734276653, -0.9734638267394214, -0.8377518803521959, -0.6079865041315164, -0.7892554160954329, -0.7029306509076686, -0.4156382497371664, -0.32928801355991855, -0.0900076363805482, -0.5115108263388009, -0.19547541176758035, -0.26245899543675943, 0.1780891720316039, -1.0985621988412326, -0.993148910000362, -1.0411691536950098, -0.5332101848766937, -0.5138413497168342, 1.056501244286741, -0.913463028024458, -0.8598467519048665, -0.5404268792247966, -0.5407232577132831, -0.8889169903918658, -0.8348558364217494, -0.8003866758197368, -0.7439215684435401, -0.6769003767754418, -0.5599334232253275, -0.49291223155722913, -0.0270667192320697, -0.0347263684600026, -0.006004901084198196, 0.23910609143859848, 0.30612580495406727, 0.8059198249984953, -0.44089136913980703, -0.1475282005307248, -0.272305616512959, -0.2723022304385234, -0.18804947500569716, 0.812391934754068, 0.8890066550712011, 1.1081803133729804, 1.1455590037144057, -0.5685784736954055, -0.6661943774827447, -0.5631529262105197, -0.6607688299978587, -0.5401293546579158, -0.45775195010414044, -0.4921957178180484, -0.5804222376740021, -0.6418369452149933, 0.20361920273282197, 0.04100133484069812, 0.012301020913566324, 0.684962606087719, 0.9182529909555962, 0.7289873611105289, 0.9335540613736576, 0.9384366837243047, 1.1525779455223495, 0.761558719488219, 0.9474362795476845, 0.8808198327020965, 1.269527925961639, 0.9723325848243177])
assert_equal(df['y'].values.tolist(),
             [0.036673509974993405, 0.41949787471257477, 0.41949787471257477, 0.09463859681545914, 0.540523880203657, 0.2602531306453612, 0.5736467869696374, 0.7277956992267002, 1.3590418647091347, -4.17116882921232e-15, 0.4105801690448108, 0.47364108769542734, 0.9889570794969303, 1.0061555118561893, 1.4469449634342366, 2.2361619150313468, 3.580824533934898, 3.0158241819844243, -1.0263170056224704, -0.8805762158521566, -0.8449053931811007, -0.9726833758206329, -0.8701297606413474, -0.6688444041403895, -0.8889843383389054, -0.8300000851364095, -0.7131781408887017, -0.5922795311928732, -0.5460348575157544, -0.030973657304758904, -0.8571353895254628, -0.8092345705100449, -0.9947228483995353, -0.8507655997627742, -0.7743281226105118, -0.7531804205983857, -0.7531804205983857, -0.6767429434461233, -0.5238679891415983, -0.555716937955041, -0.37099303483707324, -0.03339417741458055, -0.3646232450743847, -0.8181522761778088, -4.17116882921232e-15, -4.17116882921232e-15, -0.2750640010109838, 2.4259816499594655, 2.8463877742969093, 2.903715882161106, -1.0207115906313045, -0.9060553749029108, -0.8168783182252711, -0.8296178977506482, -0.7404408410730087, -0.2881857679221222, -0.17352955219372845, 0.05578287926305903, 0.3105744697706007, -0.555716937955041, -0.6003054662938607, -0.332774296260942, -0.3773628245997618, -0.30729513721018786, -0.24996702934599097, 0.6462623902642869, 0.6544157211605282, 1.5726846133497083, 1.9161436773538745, 1.906971180095603, 2.343174383044514, 2.672365117980258, 2.783454251441546, 3.5355990266198094, 4.101236357546552, 0.41988006209833606, -0.995996806352073, -0.8940801701490563, -0.8329301884272463, -0.7029864772684, -0.41379802204234023, -0.5997958831128457, -0.07365124871477213, 0.21171533265367454, 0.16330493045724162, -0.7921635339460397, -0.6392885796415146, -0.5004271628149044, -0.5004271628149044, -0.9819832688741582, -0.7781499964681249, -0.8354781043323217, -0.8099989452815676, -0.7463010476546821, -0.7526708374173706, -0.6889729397904852, -0.7271916783666165, -0.6634937807397311, -0.6316448319262884, -0.5424677752486488, -0.4660302980963863, 0.03718309315600849, 0.15183930888440225, 0.03718309315600849, 0.5085475355949606, 0.8270370237293877, 0.6614224898994856, -0.16652278345477106, -0.0009082496248689879, -0.09772905401773482, 0.08317297524261977, 0.3022937430791056, 0.4704561928140831, 0.44434005478706007, 0.492750456983493, 0.43605932809556497, 0.6042217778305424, 0.6297009368812966, -0.9726833758206329, -0.6688444041403895, -0.8889843383389054, -0.8300000851364095, -0.7131781408887017, -0.5460348575157544, -0.05645281635551307, 1.1224678729228823, 2.4613976810400136, 2.65249137392067, 3.034678759681982, -4.17116882921232e-15, -0.49838883009084406, -0.4219513529385816, -0.1728925732174596, -0.13212591873625296, 0.23350001364206935, 0.2933760374113416, 0.6297009368812966, 0.689576960650569, -1.0305210668658449, -0.7840102030497983, -0.7139425156602244, -0.774710309996273, -0.6920304388765758, -0.4136706262470865, -0.5062873693965779, -0.24818348821243819, -0.7317779269957523, -0.38335042697668903, -0.6617102396061783, -0.19276631727704785, -1.0012200339574775, -0.8750981966562444, -0.8559888273681788, -0.8012086354090574, -0.676360756060362, -0.5642524562370436, -0.798660719503982, -0.7655378127380015, -0.676360756060362, -0.6903742935382767, -0.6967440833009653, -0.6177586902436274, -0.5031024745152336, -0.6559774288197586, -0.6330461856740799, -0.49800664270508277, -0.4674316518441778, -0.6061656728755342, -0.45456467652354693, -0.4099761481847271, -0.25582723592766443, -0.21123870758884464, 0.5684235593642328, -0.5425951710439025, -0.3196525293498036, -0.4101035439799809, -0.29417337029904944, -0.24958484196022965, 0.42688683083729345, 0.3555451854951818, 0.3163072805570204, 0.32395102827224664, -0.6920304388765758, -0.6665512798258216, -0.6640033639207462, -0.638524204869992, -0.6003054662938607, -0.4729096710400899, -0.4092117734132045, -0.20537850100717117, -0.4111227103420111, 0.011194350924239241, 0.08126203831381319, -0.11683842330580045, -0.0340311563908494, 0.02648184635469174, 0.3538890401568828, 0.42140881164138133, 0.6640978015998148, 0.7316175730843133, 0.4634494240751257, 0.7437201736334216, 1.0545659140526225, 1.1800507723775866, 1.1997971206419211])

In [ ]:
# Let's Take a look at our transformed features and labels.
ax = sns.regplot(data = df, x = 'x', y = 'y', \
                 fit_reg=True, color='black', label="Data")

# Problem 5: Bayesian Modeling
Generate Traces from the liklihood of the expected value. Use the df created from problem 4, x column as x and y column as observed value. When defining the slope, intercept, and sigma set mu and the sd to be the mean and std of the transformed features, The number of samples generated should be the length of your dataset lastly save your generated traces in a variable called trace.

In [ ]:
np.random.seed(23)

In [ ]:
# YOUR CODE HERE

In [ ]:
for ans, sol in zip(trace['Intercept'].tolist(), [2.599546594244269e-17, 2.599546594244269e-17, 2.599546594244269e-17, 2.599546594244269e-17, 2.599546594244269e-17, 2.599546594244269e-17, 2.599546594244269e-17, 2.599546594244269e-17, 2.599546594244269e-17, 2.599546594244269e-17, 2.599546594244269e-17, 2.5995465942443064e-17, 2.599546594244346e-17, 2.5995465942445086e-17, 2.59954659424519e-17, 2.599546594246217e-17, 2.599546594247147e-17, 2.5995465942560062e-17, 2.5995465942811357e-17, 2.5995465943044928e-17, 2.599546594227715e-17, 2.5995465938535365e-17, 2.5995465952330902e-17, 2.599546600316783e-17, 2.5995465907416934e-17, 2.599546643565351e-17, 2.5995464672930377e-17, 2.5995466640498434e-17, 2.5995479744510793e-17, 2.5995410104587133e-17, 2.5995190703319676e-17, 2.5994149071201342e-17, 2.598834863521648e-17, 2.5989088165438664e-17, 2.648277528266454e-17, 2.6098972384893473e-17, 2.6464019594846043e-17, 1.943511527254933e-17, 8.647953028130227e-18, 2.4544452271552704e-17, 9.848239181468995e-17, 2.365786886486605e-16, 1.4643438400671032e-16, 5.381878383334833e-16, -1.0188364967793524e-14, 9.008243339701956e-14, 1.4548398460795649e-13, 2.3591260428002545e-11, 2.2553558935938508e-11, 1.1873921285876555e-11, 2.150678790212527e-11, 7.642799610590681e-11, -1.785682559657207e-10, 8.606019893310421e-10, 8.848329451832204e-09, 7.308032900184325e-09, 5.528176536536809e-08, 1.4412279083337042e-07, 7.93806315445236e-06, 1.0215590434815516e-05, -2.0510988402477596e-05, 1.1859375544602285e-05, -5.774376089426848e-05, 0.000570265744386937, -0.0010306467880735384, 0.00259525690824341, -0.005266627808393467, 0.02837161123466013, 0.02837161123466013, 0.01724925167554767, 0.011724768600422337, 0.00885058870476237, 0.04286671515932315, -0.02944664186571972, -0.013002965103081849, 0.015240666615569682, -0.01585913910126165, -0.010632160067414872, -0.02060184379989701, 0.01884682354041577, 0.04257048821126523, -0.0036258568231731275, 0.014715336240292826, -0.022493159490173165, -0.022493159490173165, -0.04253736404308026, 0.005426382879620593, -0.0017817829519366686, -0.014176825792164135, 0.015614417773033174, 0.015614417773033174, 0.010199968808349193, -0.00611981294137143, 0.008217597898956016, -0.008385406867808627, -0.026910592159422927, 0.03976165659610385, 0.05899961482549553, 0.06050109647728707, 0.06050109647728707, 0.058773929714206086, -0.007844904981178685, 0.027784425396176548, -0.040567188032123384, -0.040567188032123384, -0.023646032082837564, -0.03605196201652165, 0.06544556601093651, 0.1097959077651095, -0.10495858347897319, -0.10495858347897319, -0.13102691367351682, -0.10393455566411682, -0.08442633196456965, 0.011341186347319733, 0.0019627559903465687, 0.03392247275189204, 0.03392247275189204, -0.031055516917157473, 0.03033130668785704, -0.001067116692924279, -0.02190016160048078, 0.12221819669993488, -0.11122674717492291, -0.04662625750419578, 0.04318527280399242, 0.04318527280399242, -0.01776540031356123, 0.03972874994870287, 0.02945594669088527, 0.02945594669088527, 0.00440041254624909, -0.002149954977177328, -0.04190145190002927, -0.030155179021398602, -0.050582977055468575, -0.050582977055468575, 0.06223166655804818, 0.03005726300946618, 0.027935243901087148, -0.06856403582425999, -0.06856403582425999, -0.027835857855982443, -0.06228830461715793, -0.023368767643711482, -0.006045629140772737, -0.006045629140772737, -0.004159758851684709, -0.012831056549133921, -0.02072333093593773, -0.029330507225535228, 0.06195800163853905, -0.07136853893313444, -0.07637960524120928, 0.06278715706509136, 0.06278715706509136, 0.0052359452196775, 0.014531616601715664, 0.013147659051835505, 0.01873772680263413, 0.007220804353747756, 0.007220804353747756, 0.007304843196371105, 0.007278911604668809, 0.017803373867567883, 0.025732586928824148, 0.025732586928824148, 0.04299613479273092, 0.05829711174413636, 0.006226735880642509, 0.02462491289323079, 0.02462491289323079, -0.030154632576806582, -0.03282589257522809, 0.013182236040075544, -0.031895084767526005, -0.017844697929158743, -0.017844697929158743, 0.03605179114670963, 0.0494604881036421, 0.02892822193144939, 0.02202768040149972, 0.025405072187404718, -0.0005486722425445145, -0.041161985432629694, 0.08082733534949522, 0.005384339956009655, -0.01475879867641053, 0.005993293899996991, 0.005993293899996991, -0.03833627681906744, 0.08630867040678526, 0.08630867040678526, -0.007782424148052403, -0.019169958327831946, 0.011928609345365956, 0.03765107601411058, 0.0012093006611055392, 0.04205250282070072, 0.04092569402156641, 0.04092204312001877, 0.04092204312001877, 0.05062427826381396, 0.03400990712148139, 0.007698250500777654]):
    assert_almost_equal(ans,sol,places=1)
for ans, sol in zip(trace['sigma'].tolist(), [4.4003497944458566e-14, 4.4003497944458566e-14, 4.4003497944458566e-14, 4.4003497944458566e-14, 4.4003497944458566e-14, 4.4003497944458566e-14, 4.4003497944458566e-14, 4.4003497944458566e-14, 4.4003497944458566e-14, 4.4003497944458566e-14, 4.4003497944458566e-14, 4.400815395696074e-14, 4.401530540920153e-14, 4.40311867731488e-14, 4.407892565403599e-14, 6.251535143669991e-14, 6.282282866801616e-14, 3.433996352610196e-13, 3.4346875250637173e-13, 3.48057108248631e-13, 3.640097739184625e-13, 3.958713386323943e-13, 9.90662581550154e-13, 1.507045411655484e-12, 2.8940865534554915e-12, 3.667195490145491e-12, 1.34104691387065e-11, 1.4515093275126889e-11, 2.5331187665084456e-11, 5.112671967503548e-11, 9.156950967807371e-11, 2.9834189480675374e-10, 6.52915608091675e-10, 8.151489554247179e-10, 6.918213389847465e-09, 9.226518467840112e-09, 9.378334737393676e-09, 2.3206286798515954e-08, 2.3641015947520927e-08, 2.9161984642284747e-08, 3.5064478317716335e-07, 3.509469052940717e-07, 3.549477457829567e-07, 4.060232979333619e-07, 8.541245574162416e-07, 2.3507898274654845e-06, 2.7413349377595072e-06, 9.057269403614557e-05, 9.057381177930018e-05, 9.063073699353343e-05, 9.080944177488275e-05, 9.302613121934488e-05, 0.00011854202137796414, 0.000269849483998077, 0.0004258956085284293, 0.0014833164032518316, 0.0016187641039432554, 0.0028543355380578183, 0.030777423361164503, 0.030800795877507083, 0.03722875828982028, 0.03906103493887553, 0.05695443174452383, 0.44536889853789546, 0.4490150721022846, 0.49585074603673196, 0.5828554646674956, 1.048596623725216, 1.048596623725216, 0.9419651734528015, 0.8885961045049285, 0.8411283712550514, 0.7489735390304998, 0.5909441795990836, 0.538339132785892, 0.5476299364755798, 0.5558821569536296, 0.6264989053801938, 0.582768684381172, 0.6150534240427339, 0.5453010134597049, 0.6011431408949709, 0.6002333603089037, 0.6129103038319907, 0.6129103038319907, 0.5408291294469351, 0.6365923052176073, 0.6571701635683225, 0.572054917629928, 0.580181719080576, 0.580181719080576, 0.6073779884966581, 0.5411465626438172, 0.6203802128970385, 0.6218772307302167, 0.521299418384763, 0.5413655797470076, 0.5974239535641871, 0.5940009083666714, 0.5940009083666714, 0.6061866638441435, 0.5365918532666379, 0.566415570776852, 0.5855486764071013, 0.5855486764071013, 0.5200378554088111, 0.5805246092575738, 0.581924476245852, 0.6153699431711952, 0.5824986068963219, 0.5824986068963219, 0.5961642840879618, 0.574143631112704, 0.555273756628975, 0.6265859839619591, 0.6182899830792009, 0.5443398126130066, 0.5443398126130066, 0.5904124766527675, 0.551430796129449, 0.5408619530772213, 0.6381457655066253, 0.5313107614444502, 0.5794001801165662, 0.5991519206911721, 0.5539995861329906, 0.5539995861329906, 0.5522236332022572, 0.6031849142926197, 0.5631679689597828, 0.5631679689597828, 0.5615320083389964, 0.5444881131048552, 0.5924437396124059, 0.5212114851458504, 0.5643556409894793, 0.5643556409894793, 0.6106578657169808, 0.5949493854064855, 0.5573810961312364, 0.5880438929469863, 0.5880438929469863, 0.6550763878615983, 0.6587499561196936, 0.6164168380332182, 0.545338319636245, 0.545338319636245, 0.5386015870371266, 0.5549565656665393, 0.5830971562950404, 0.5675906839613574, 0.5936030087667994, 0.5775522185786193, 0.5682488453078983, 0.5577919733736073, 0.5577919733736073, 0.5571236976548698, 0.5906922413917157, 0.5658173480075022, 0.5457097429592703, 0.551969122445228, 0.551969122445228, 0.5872185906323301, 0.603091373652094, 0.6264408457602615, 0.6361898549159836, 0.6361898549159836, 0.6528712183079571, 0.5343050432406214, 0.6158608686863952, 0.5672345149945837, 0.5672345149945837, 0.5712619416543354, 0.5846249397191706, 0.5794168559635499, 0.5653958426202413, 0.5925590494572308, 0.5925590494572308, 0.5899062698705996, 0.5882246593392032, 0.5477853597938634, 0.584744839440813, 0.5860686484196571, 0.5922600081889854, 0.5632758782589891, 0.5732066700123455, 0.5941097585477875, 0.5662602509335251, 0.5832710580417607, 0.5832710580417607, 0.5734226041978012, 0.5741094256713061, 0.5741094256713061, 0.5673557403862939, 0.5691029468926494, 0.5679626983643765, 0.6032077106940534, 0.5834123161660331, 0.5942009370839112, 0.6066613653336348, 0.5484437209595103, 0.5484437209595103, 0.5777856934154011, 0.5767987514114554, 0.5387953917229518]):
    assert_almost_equal(ans,sol,places=1)

In [ ]:
# Let's visualize our traces from the likelihood
pm.traceplot(trace)

# Problem 6: General Linear Model
With the df created from problem 4, fit a general linear model using our transformed features and labels and generate x number of samples where x is the length of the the dataframe. Save your traces in a variable called trace and save your model as model_glm.

In [ ]:
# YOUR CODE HERE

In [ ]:
assert_equal(205, len(trace))
assert_equal(model_glm.missing_values, [])
for t in trace:
    for key in t:
        assert_true(key in ['Intercept', 'sd', 'sd_log_', 'x'])

In [ ]:
# Let's visualize our traces from the likelihood
pm.traceplot(trace)